In [1]:
import os
import pandas as pd
import numpy as np

import torch
import torchvision

from PIL import Image
import pillow_heif

from sklearn.metrics.pairwise import cosine_similarity

from glob import glob
import pickle

import cv2
from collections import defaultdict
from facenet_pytorch import MTCNN
import dlib

c:\Users\Senne\AI_Frameworks\Project\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
test_data = glob('testset/testset/*.jpg')
source_folder = "Testset/Testset"
output_folder = "Testset/Testset"

def list_extensions(folder_path):
    extensions = set()
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        if os.path.isfile(file_path):
            _, extension = os.path.splitext(filename)
            extensions.add(extension.lower())
    return extensions

def convert_to_jpg(file_path, source_folder):
    try:
        _, extension = os.path.splitext(file_path)
        extension = extension.lower()

        if extension == ".heic":
            heif_file = pillow_heif.open_heif(file_path)
            image = Image.frombytes(
                heif_file.mode, heif_file.size, heif_file.data, "raw"
            )
        else:
            image = Image.open(file_path)

        image = image.convert("RGB")

        base_name = os.path.basename(file_path)
        new_file_path = f"{source_folder}{os.path.splitext(base_name)[0]}.jpg"
        os.remove(file_path)
        image.save(new_file_path, "JPEG")
        print(f"Converted {file_path} to {new_file_path}")

        return new_file_path
    except Exception as e:
        print(f"Failed to convert {file_path}: {e}")
        return None

def convert_images_from_df(df, source_folder):
    new_images = []
    for _, row in df.iterrows():
        file_path = row["File Path"]
        converted_path = convert_to_jpg(file_path, source_folder)
        if converted_path:
            new_images.append(converted_path)
    return new_images


def extract_frame_from_mp4(video_path, output_folder, frame_number=0):
    try:
        video = cv2.VideoCapture(video_path)
        
        video.set(cv2.CAP_PROP_POS_FRAMES, frame_number)
        success, frame = video.read()
        video.release()

        if not success:
            print(f"Failed to extract frame from {video_path}")
            return None

        base_name = os.path.basename(video_path)
        output_file = os.path.join(output_folder, f"{os.path.splitext(base_name)[0]}.jpg")
        
        os.remove(video_path)
        cv2.imwrite(output_file, frame)
        print(f"Extracted frame from {video_path} and saved to {output_file}")
        return output_file
    except Exception as e:
        print(f"Error processing {video_path}: {e}")
        return None

def process_mp4_to_jpg(mp4_df, output_folder):
    jpg_files = []
    for _, row in mp4_df.iterrows():
        video_path = row["File Path"]
        jpg_path = extract_frame_from_mp4(video_path, output_folder)
        if jpg_path:
            jpg_files.append(jpg_path)
    return jpg_files

def list_non_jpg_images(folder_path, image_extensions):
    non_jpg_files = []
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        if os.path.isfile(file_path):
            _, extension = os.path.splitext(filename)
            if extension.lower() in image_extensions:
                non_jpg_files.append(file_path)
    return non_jpg_files

extensions_in_folder = list_extensions(source_folder)

extensions_in_folder.discard(".jpg")
if ".mp4" in extensions_in_folder:
    mp4_Extension = ".mp4"

print("Extensions found in the folder:")
for ext in sorted(extensions_in_folder):
    print(ext if ext else "No extension (e.g., hidden or extensionless files)")

non_jpg_images = list_non_jpg_images(source_folder, extensions_in_folder)
img_not_jpg_df = pd.DataFrame(non_jpg_images, columns=["File Path"])

if ".mp4" in extensions_in_folder:
    mp4_images = list_non_jpg_images(source_folder, mp4_Extension)
    if mp4_images:
        mp4_img_df = pd.DataFrame(mp4_images, columns=["File Path"])
    
convert_images_from_df(img_not_jpg_df, output_folder)
if ".mp4" in extensions_in_folder:
    jpg_files = process_mp4_to_jpg(mp4_img_df, output_folder)
    mp4_to_jpg_df = pd.DataFrame(jpg_files, columns=["image"])

Extensions found in the folder:


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"PyTorch version: {torch.__version__}")
print(f"Torchvision version: {torchvision.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"CUDA device: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'None'}")
print(f"Using device: {device}")
mtcnn = MTCNN(keep_all=True, device=device)

PyTorch version: 2.2.2+cu118
Torchvision version: 0.17.2+cu118
CUDA available: True
CUDA device: NVIDIA GeForce RTX 3050 Ti Laptop GPU
Using device: cuda


In [6]:
def detect_faces_mtcnn(img, img_index, cropped_faces):
    try:
        correct_rotation = 0
        highest_prob = 0.0
        confidence_threshold = 0.82
        length = 4
        horizontal_preffered_dimensions = (1008, 756)
        vertical_preffered_dimensions = (756, 1008)
        valid_boxes = []
        best_probs = []
        found_landmarks  = []
        #print(img.shape)
        for i in range(length):
            empty_box = False
            degrees = 90 * i
            rotated_img = img

            if degrees == 90:
                rotated_img = cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE)
            elif degrees == 180:
                rotated_img = cv2.rotate(img, cv2.ROTATE_180)
            elif degrees == 270:
                rotated_img = cv2.rotate(img, cv2.ROTATE_90_COUNTERCLOCKWISE)

            height, width, _ = rotated_img.shape

            if width > height: 
                scale = min(horizontal_preffered_dimensions[0] / width, horizontal_preffered_dimensions[1] / height)
            else:
                scale = min(vertical_preffered_dimensions[0] / width, vertical_preffered_dimensions[1] / height)

            new_width = int(width * scale)
            new_height = int(height * scale)

            resized_img = cv2.resize(rotated_img, (new_width, new_height))
            #resized_img = cv2.resize(rotated_img, (width, height))
            #print(resized_img.shape)
                
            boxes, probs, landmarks = mtcnn.detect(resized_img, landmarks=True)

            if boxes is None or probs is None or landmarks is None:
                empty_box = True
                boxes, landmarks = [], []

            #print(f"Image index {img_index}: Detection probabilities - {probs}")
            rotated_img = cv2.cvtColor(rotated_img, cv2.COLOR_BGR2RGB)

            if not empty_box:
                filtered = [
                    (box, prob, landmark)
                    for box, prob, landmark in zip(boxes, probs, landmarks)
                    if prob >= confidence_threshold
                    #if Valid_Landmarks(box, landmark)
                ]
                boxes, probs, landmarks = zip(*filtered) if filtered else ([], [], [])
                boxes = sorted(boxes, key=lambda box: box[0])

                for prob in probs:
                    if prob > highest_prob:
                        highest_prob = prob
                        best_probs = probs
                        valid_boxes = boxes
                        found_landmarks = landmarks
                        correct_rotation = degrees

        if correct_rotation == 90:
            img = cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE)
        elif correct_rotation == 180:
            img = cv2.rotate(img, cv2.ROTATE_180)
        elif correct_rotation == 270:
            img = cv2.rotate(img, cv2.ROTATE_90_COUNTERCLOCKWISE)
            
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        for box, landmark, prob in zip(valid_boxes, found_landmarks, best_probs):
            #print(f"Image index {img_index}: Detection probabilities - {prob}")    
            
            height, width, _ = img.shape
            x1, y1, x2, y2 = [int(coord) for coord in box]
            
            x1 = int(x1 / scale)
            y1 = int(y1 / scale)
            x2 = int(x2 / scale)
            y2 = int(y2 / scale)

            x1 = max(0, min(x1, width - 1))
            y1 = max(0, min(y1, height - 1))
            x2 = max(0, min(x2, width - 1))
            y2 = max(0, min(y2, height - 1))

            cropped_face = img[y1:y2, x1:x2]
            cropped_faces.append(cropped_face)

            #cropped_face = cv2.cvtColor(cropped_face, cv2.COLOR_BGR2RGB)

            #cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)

            #print(f"Face landmarks: {landmark}")
            for point in landmark:
                px, py = int(point[0]), int(point[1])
                #cv2.circle(img, (px, py), 5, (255, 0, 0), -1)  # Draw landmarks

            #plt.imshow(cropped_face)
            #plt.axis("off")
            #plt.show()

        #img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        #plt.imshow(img)
        #plt.axis("off")
        #plt.show()

        Cropped_FilePath = "CroppedTestImg"
        os.makedirs(Cropped_FilePath, exist_ok=True)

        return len(valid_boxes)
    except Exception as e:
        print(f"Error detecting faces in image index {img_index}: {e}")
        return 0

In [7]:
start = 0
Cropped_FilePath = "CroppedTestImg"
CroppedImg_filepaths = []
bad_detect = pd.DataFrame()

bad = 0
good = 0

length = len(test_data)
for i in range(length):

    cropped_faces = []
    position = i + start
    img = cv2.imread(test_data[position])
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    detected = detect_faces_mtcnn(img, position, cropped_faces)

    img_name = os.path.basename(test_data[position])
    img_name = os.path.splitext(img_name)[0]

    for d in range(detected):
        Img_FilePath = os.path.join(Cropped_FilePath, f"{img_name}_{d}.jpg")
        CroppedImg_filepaths.append(Img_FilePath)
        cv2.imwrite(Img_FilePath, cropped_faces[d])
        good += 1

    if detected == 0:
        bad_detect = pd.concat([bad_detect, pd.DataFrame({
            'image': [test_data[position]],
            'guess': ['nothing']
        })], ignore_index=True)
        bad += 1

    print(f"Processed {i+1}/{length} images")

print(f"faces found: {good}")
print(f"None faces found: {bad}")


Processed 1/99 images
Processed 2/99 images
Processed 3/99 images
Processed 4/99 images
Processed 5/99 images
Processed 6/99 images
Processed 7/99 images
Processed 8/99 images
Processed 9/99 images
Processed 10/99 images
Processed 11/99 images
Processed 12/99 images
Processed 13/99 images
Processed 14/99 images
Processed 15/99 images
Processed 16/99 images
Processed 17/99 images
Processed 18/99 images
Processed 19/99 images
Processed 20/99 images
Processed 21/99 images
Processed 22/99 images
Processed 23/99 images
Processed 24/99 images
Processed 25/99 images
Processed 26/99 images
Processed 27/99 images
Processed 28/99 images
Processed 29/99 images
Processed 30/99 images
Processed 31/99 images
Processed 32/99 images
Processed 33/99 images
Processed 34/99 images
Processed 35/99 images
Processed 36/99 images
Processed 37/99 images
Processed 38/99 images
Processed 39/99 images
Processed 40/99 images
Processed 41/99 images
Processed 42/99 images
Processed 43/99 images
Processed 44/99 imag

In [10]:
df_embeddings = pd.read_pickle("embeddings_C_54.pkl")

with open("Labels_faces.pickle", 'rb') as f:
    P_name = pickle.load(f)
    P_name = {n: i for i, n in P_name.items()}

dir = "CroppedTestImg"

shape_predictor_path = "shape_predictor_68_face_landmarks.dat"
face_rec_model_path = "dlib_face_recognition_resnet_model_v1.dat"

sp = dlib.shape_predictor(shape_predictor_path)
facerec = dlib.face_recognition_model_v1(face_rec_model_path)

results = pd.DataFrame()
results = pd.concat([results, bad_detect], ignore_index=True)

for image_file in os.listdir(dir):
    img_path = os.path.join(dir, image_file)

    img = cv2.imread(img_path)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    h, w, _ = img.shape
    rect = dlib.rectangle(0, 0, w, h)

    try:
        shape = sp(img_rgb, rect)
    except Exception as e:
        print(f"Error processing {image_file}: {e}")
        continue

    face_chip = dlib.get_face_chip(img_rgb, shape)
    face_descriptor = facerec.compute_face_descriptor(face_chip)

    embedding = np.array(face_descriptor).reshape(1, -1)

    similarities = []
    for stored_embedding in df_embeddings['embedding']:
        stored_embedding = np.array(stored_embedding).reshape(1, -1)
        similarity = cosine_similarity(embedding, stored_embedding)
        similarities.append(similarity[0][0])

    max_similarity_index = np.argmax(similarities)

    similarity_map = {}
    for idx, sim in enumerate(similarities):
        stored_id = df_embeddings.iloc[idx]['id']
        name = P_name.get(stored_id, "Unknown")
        if name not in similarity_map or sim > similarity_map[name]:
            similarity_map[name] = sim
    
    best_match_id = df_embeddings.iloc[max_similarity_index]['id']
    predicted_label = P_name.get(best_match_id, "Unknown")
    confidence = similarities[max_similarity_index]

    results = pd.concat([results, pd.DataFrame({
        'image': [img_path],
        'guess': [predicted_label],
        'confidence': confidence,
    })], ignore_index=True)

    print(f"Processed: {image_file} - Predicted: {predicted_label} - Similarity: {confidence:.4f}")

results = results.sort_values(by='image', key=lambda col: col.map(os.path.basename)).reset_index(drop=True)

print(results)


Processed: 0037_0.jpg - Predicted: Senne - Similarity: 0.9742
Processed: 0039_0.jpg - Predicted: Konrad - Similarity: 0.9121
Processed: 0039_1.jpg - Predicted: Youssef - Similarity: 0.9800
Processed: 0039_2.jpg - Predicted: Konrad - Similarity: 0.9290
Processed: 0040_0.jpg - Predicted: Seppe - Similarity: 0.9763
Processed: 0040_1.jpg - Predicted: Alper - Similarity: 0.9626
Processed: 0040_2.jpg - Predicted: Daiane - Similarity: 0.9412
Processed: 0040_3.jpg - Predicted: Senne - Similarity: 0.9309
Processed: 0043_0.jpg - Predicted: Raul - Similarity: 0.9710
Processed: 0051_0.jpg - Predicted: Lasse - Similarity: 0.9830
Processed: 0051_1.jpg - Predicted: Akif - Similarity: 0.9585
Processed: 0058_0.jpg - Predicted: Bart - Similarity: 0.9887
Processed: 0058_1.jpg - Predicted: Seppe - Similarity: 0.9405
Processed: 0062_0.jpg - Predicted: Konrad - Similarity: 0.9797
Processed: 0066_0.jpg - Predicted: Nelli - Similarity: 0.9269
Processed: 0066_1.jpg - Predicted: Florian - Similarity: 0.9607
Pro

In [13]:
grouped_data = defaultdict(list)

for i in range(len(results)):
    
    person_path = results['image'][i]
    guess = results['guess'][i]

    file_name = os.path.splitext(os.path.basename(person_path))[0]
    prefix = file_name.split('_')[0]
    
    grouped_data[prefix].append(guess)

FinalResaults = pd.DataFrame()
for file_path in test_data:
    base_name = os.path.splitext(os.path.basename(file_path))[0]
    if base_name in grouped_data:
        label_name = grouped_data[base_name]
        ordered_label = list()
        img_name = base_name.lstrip("0")
        concatenated_labels = ";".join(label.lower().replace(" ", "") for label in label_name)
        FinalResaults = pd.concat([FinalResaults, pd.DataFrame({
            "image": [img_name],
            "label_name": [concatenated_labels],
        })], ignore_index=True)

output_csv_path = "FinalResults_testing.csv"
FinalResaults.to_csv(output_csv_path, index=False)

print(f"Final results saved to {output_csv_path}")


0037
0039
0039
0039
0040
0040
0040
0040
0043
0045
0051
0051
0058
0058
0062
0066
0066
0078
0103
0125
0139
0160
0160
0160
0160
0161
0161
0166
0168
0174
0179
0179
0182
0182
0182
0182
0194
0194
0221
0226
0233
0233
0233
0245
0245
0246
0246
0246
0246
0255
0255
0255
0262
0262
0262
0268
0275
0282
0282
0316
0344
0344
0377
0377
0377
0381
0381
0381
0381
0398
0398
0398
0398
0416
0418
0418
0418
0420
0420
0423
0423
0427
0435
0443
0444
0448
0450
0452
0452
0452
0452
0454
0454
0454
0472
0472
0472
0481
0513
0521
0521
0521
0521
0540
0540
0551
0551
0584
0584
0589
0590
0592
0594
0594
0596
0599
0603
0603
0603
0605
0605
0612
0612
0614
0615
0615
0621
0624
0624
0629
0638
0638
0638
0641
0646
0646
0648
0664
0664
0668
0668
0668
0669
0669
0669
0669
0675
0675
0676
0676
0679
0679
0698
0698
0754
0754
0757
0757
0763
0764
0765
0766
0770
0772
0779
0779
0779
0779
0786
0786
0790
0792
0794
0799
0805
0808
0818
0818
0818
0822
0822
0825
Final results saved to FinalResults_testing.csv
